In [1]:
import time, os, random, re, requests
from io import BytesIO
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import urllib.request
from selenium import webdriver
from selenium.webdriver.common.by import By
from PIL import Image

from selenium.common.exceptions import NoSuchElementException,WebDriverException
import warnings
warnings.filterwarnings('ignore')

In [22]:
#제품코드 -> 문자로 읽어옴
df = pd.read_excel(r'D:\ODOC\jupyter_notebook\제품데이터_FULL\화장품 통합 데이터_유니크_V3.xlsx',
                  dtype={'product_code':str})

#가격 -> 없는 경우 세일가=가격으로 채우기
#소수점 삭제
idx = df.price.isna()
df.loc[idx, 'price'] = df.loc[idx, 'sale_price']
df.price = df.price.apply(round)

In [23]:
df.head()

,product_code,category,category_modify,image_path,brand_name,product_name,price,sale_price,review_num,review_star,ingr_name
0,000000001,화장품/향수-스킨케어-에센스/세럼/오일,에센스/앰플/세럼/오일,img/000000001.png,스킨레지민,1.5 레티놀 부스터,145000,145000,0,NaN,정제수|글리세린|카프릴릭/카프릭트라이글리세라이드|시어버터|글리세릴스테아레이트시트레이...
1,000000002,스킨케어/오일,에센스/앰플/세럼/오일,img/000000002.png,닥터올가,100 퓨어 로즈힙 오일,39200,39200,0,NaN,로즈힙열매오일|해바라기씨오일|로즈마리잎추출물
2,000000003,스킨케어/오일,에센스/앰플/세럼/오일,img/000000003.png,닥터올가,100 퓨어 타마누 라이트 오일,26400,26400,0,NaN,해바라기씨오일|로즈마리잎추출물
3,000000004,스킨케어/오일,에센스/앰플/세럼/오일,img/000000004.png,닥터올가,100 퓨어 타마누 오일,42400,42400,0,NaN,해바라기씨오일|로즈마리잎추출물
4,000000005,스킨케어/오일,에센스/앰플/세럼/오일,img/000000005.png,닥터올가,100 퓨어 햄프씨드 오일,34400,34400,0,NaN,삼씨추출물|해바라기씨오일|로즈마리잎추출물


In [1]:
#화장품 사이트: 제품 카테고리 코드
categories = {'스킨케어':{'스킨/토너': '100000100010013',
                  '에센스/세럼/앰플': '100000100010014',
                  '크림': '100000100010015',
                  '로션': '100000100010016',
                  '미스트/오일': '100000100010010'},
             '마스크팩':{'시트팩': '100000100090001',
                  '패드': '100000100090004',
                  '페이셜팩': '100000100090002',
                  '코팩': '100000100090005',
                  '패치': '100000100090006'},
             '클렌징':{'클렌징폼/젤': '100000100100001',
                  '오일/밤': '100000100100004',
                  '워터/티슈/밀크': '100000100100005',
                  '필링/스크럽': '100000100100007',
                  '립&아이리무버': '100000100100006'},
             '선케어':{'선블록': '100000100110001', 
                     '태닝/애프터선': '100000100110002'},
             '메이크업':{'립메이크업': '100000100020006',
                  '베이스메이크업': '100000100020001',
                  '아이메이크업': '100000100020007'},
             '더모코스메틱':{'스킨케어': '100000100080013',
                  '바디케어': '100000100080004',
                  '클렌징': '100000100080006',
                  '선케어': '100000100080005',
                  '마스크팩': '100000100080011'},
             '맨즈케어':{'스킨케어': '100000100070007',
                  '메이크업': '100000100070008',
                  '헤어케어': '100000100070009',
                  '바디케어': '100000100070011',
                  '프래그런스/라이프': '100000100070017'},
             '향수':{'여성향수': '100000100050003',
                  '남성향수': '100000100050004',
                  '유니섹스향수': '100000100050011',
                  '고체향수': '100000100050010'},
             '헤어케어':{'샴푸/린스': '100000100040008',
                  '트리트먼트/팩': '100000100040007',
                  '헤어에센스': '100000100040013',
                  '스타일링': '100000100040011'},
             '바디케어':{'로션/오일': '100000100030014',
                  '샤워/입욕': '100000100030005',
                  '립케어': '100000100030008',
                  '핸드케어': '100000100030016',
                  '바디미스트': '100000100030015',
                  '맘&베이비': '100000100030020'}}

In [3]:
# 각 상세페이지 링크 따오기
def detail_link(num):
    href_list=[]   #제품 페이지 링크
    goodsno_list=[]   #제품 코드
    img_list=[]   #제품 이미지
    page = 0
    
    while (page != -1):   #and (page<10)  ->가져올 페이지 수 제한
        page += 1
        url = f'https://www.oliveyoung.co.kr/store/display/getMCategoryList.do?dispCatNo={num}&fltDispCatNo=&prdSort=01&pageIdx={page}&rowsPerPage=24&searchTypeSort=btn_thumb&plusButtonFlag=N&isLoginCnt=0&aShowCnt=0&bShowCnt=0&cShowCnt=0&trackingCd=Cat100000100010013_Small&amplitudePageGubun=&t_page=&t_click=&midCategory=%EC%8A%A4%ED%82%A8%2F%ED%86%A0%EB%84%88&smallCategory=%EC%A0%84%EC%B2%B4&checkBrnds=&lastChkBrnd='
        #카테고리, 페이지 바꿔가며 url 생성
        driver.get(url)
        time.sleep(random.uniform(0.1, 0.3))
        
        for n in range(2,8):
            for m in range(1,5):
                xpath = f'//*[@id="Contents"]/ul[{n}]/li[{m}]/div/a'
                xpath_img = f'//*[@id="Contents"]/ul[{n}]/li[{m}]/div/a/img'
                
                try: 
                    goodsno = driver.find_element(By.XPATH, xpath).get_attribute('data-ref-goodsno')
                    link = driver.find_element(By.XPATH, xpath).get_attribute('href')
                    img = driver.find_element(By.XPATH, xpath_img).get_attribute('src')
                    
                    goodsno_list.append(goodsno)
                    href_list.append(link)
                    img_list.append(img)

                except NoSuchElementException: 
                    page=-1
                    
    return goodsno_list, href_list, img_list

In [ ]:
## 데이터프레임 생성
# new_df = pd.DataFrame(columns=['goodsno','category','images','LINK'])

In [43]:
driver = webdriver.Chrome()
for key in categories.keys():
    for sub_cat, num in categories[key].items():
        goods, href, imgs = detail_link(num)
        cat = key+'_'+sub_cat
        
        pd_df = pd.DataFrame({'goodsno': goods, 'category': cat, 'images':imgs,'LINK': href})
        new_df = pd.concat([new_df, pd_df], ignore_index=True)
driver.quit()

In [61]:
#중복제거 & 인덱스 재배열
new_df.drop_duplicates(subset='goodsno', inplace=True)
new_df.reset_index(drop=True, inplace=True)

In [115]:
#제품번호 붙여주기 (기존 데이터 이후로)
for i in range(len(new_df)):
    new_df.product_code.iloc[i] = '{:09d}'.format(6192+i)

In [5]:
# new_df = pd.read_excel(r'D:\ODOC\jupyter_notebook\제품데이터_FULL\화장품 통합 데이터_유니크_V3_plus.xlsx')
# new_df

In [2]:
def CSS(selector):
    try:
        return driver.find_element(By.CSS, selector).text
    except: return None 
    
def XPATH(path):
    try:
        return driver.find_element(By.XPATH, path).text
    except: return None 

In [72]:
# 제품 상세정보 크롤링
def product_detail_crawling(url):
    image_list, name, brand, price, sale_price = None,None,None,None,None
    driver.get(url)
    time.sleep(random.uniform(0.5, 1))

    #상세페이지 이미지
    css_selector=[
#     '#Contents > div.tabConts.prd_detail_cont.show > div.detail_area > div.contEditor > img',
#     '#Contents > div.tabConts.prd_detail_cont.show > div.detail_area > div.contEditor > center > p > img',
#     '#Contents > div.tabConts.prd_detail_cont.show > div.detail_area > div.contEditor > div > div > img',
#     '#Contents > div.tabConts.prd_detail_cont.show > div.detail_area > div.contEditor > div > img',
#     '#Contents > div.tabConts.prd_detail_cont.show > div.detail_area > img',
#     '#Contents > div.tabConts.prd_detail_cont.show > div.detail_area > picture > img',
    '#Contents > div.tabConts.prd_detail_cont.show > div.detail_area > div.contEditor > p > img',
    '#Contents > div.tabConts.prd_detail_cont.show > div.detail_area > div.contEditor > center > img',
    '#Contents > div.tabConts.prd_detail_cont.show > div.detail_area > div.contEditor > aa > img',
    ]
    
    for css in css_selector:
        try:
            images = driver.find_elements(By.CSS_SELECTOR, css)
            if len(images)!=0:
                image_list = [img.get_attribute('src') for img in images]
                break
        except NoSuchElementException: 
            print('pass')
            pass
    
    #제품명 / 브랜드 / 가격 / 세일가
    name = XPATH('//*[@id="Contents"]/div[2]/div[2]/div/p[2]')
    brand = XPATH('//*[@id="moveBrandShop"]')
    price = XPATH('//*[@id="Contents"]/div[2]/div[2]/div/div[1]/span/strong')
    price = XPATH('//*[@id="Contents"]/div[2]/div[2]/div/div[1]/span[1]/strike')
    sale_price = XPATH('//*[@id="Contents"]/div[2]/div[2]/div/div[1]/span[2]/strong')
    
    return image_list, name, brand, price, sale_price

In [78]:
# #새 칼럼들 추가
# new_columns=['product_name','brand_name','price','sale_price','detail_images']
# new_df = pd.concat([new_df, pd.DataFrame(columns=new_columns)], axis=1)

In [92]:
from selenium.common.exceptions import TimeoutException
driver = webdriver.Chrome()
# start, end = 8407, len(new_df)

# idx = new_df[new_df.product_name.isna()].index
idx = new_df[new_df.price.isna()].index
# idx = new_df[new_df.detail_images.isna()].index[::-1]

for i in idx:
    url = new_df.LINK[i]

    try:
        data = product_detail_crawling(url)

        new_df.loc[i, 'product_name'] = data[1]
        new_df.loc[i, 'brand_name'] = data[2]
        new_df.loc[i, 'price'] = data[3]
        new_df.loc[i, 'sale_price'] = data[4]
        
        if np.isnan(new_df['detail_images'].iloc[i]):
            new_df['detail_images'].iloc[i] = data[0]
    
    except (Exception,TimeoutException) as e:
        print(e)
driver.quit()

In [ ]:
idx = new_df[new_df.price.isna()].index
new_df.drop(idx,inplace=True)   #제품페이지 삭제된 경우-> drop

In [93]:
idx = new_df[new_df.detail_images.isna()]#.index[::1]
new_df[new_df.detail_images.isna()]

,product_code,category,images,LINK,goodsno,product_name,brand_name,price,sale_price,detail_images
310,6502,스킨케어_스킨/토너,https://image.oliveyoung.co.kr/uploads/images/...,https://www.oliveyoung.co.kr/store/goods/getGo...,A000000153694,에뛰드 순정약산성5.5진정토너 더블 기획 (350ml+350ml),에뛰드,"27,000",NaN,None
551,6746,스킨케어_에센스/세럼/앰플,https://image.oliveyoung.co.kr/uploads/images/...,https://www.oliveyoung.co.kr/store/goods/getGo...,A000000201023,코스알엑스 더 비타민C 13 세럼 20ml,코스알엑스,"18,000",NaN,None
604,6800,스킨케어_에센스/세럼/앰플,https://image.oliveyoung.co.kr/uploads/images/...,https://www.oliveyoung.co.kr/store/goods/getGo...,A000000191334,바이오던스 포어 타이트닝 콜라겐 앰플 30ml,바이오던스,"35,000",NaN,None
704,6900,스킨케어_에센스/세럼/앰플,https://image.oliveyoung.co.kr/uploads/images/...,https://www.oliveyoung.co.kr/store/goods/getGo...,A000000154626,달팡 인트랄 액티브 스태빌라이징 로션 100ml,달팡,"102,000",NaN,None
721,6917,스킨케어_에센스/세럼/앰플,https://image.oliveyoung.co.kr/uploads/images/...,https://www.oliveyoung.co.kr/store/goods/getGo...,A000000131594,비욘드 엔젤아쿠아 시카 카밍 앰플 100ml,비욘드,"22,000","16,500",None
...,...,...,...,...,...,...,...,...,...,...
8639,14854,바디케어_립케어,https://image.oliveyoung.co.kr/uploads/images/...,https://www.oliveyoung.co.kr/store/goods/getGo...,A000000172129,에뛰드 진저슈가 립밤 스틱 3.7g,에뛰드,"7,000","6,300",None
8653,14868,바디케어_립케어,https://image.oliveyoung.co.kr/uploads/images/...,https://www.oliveyoung.co.kr/store/goods/getGo...,A000000172131,에뛰드 진저슈가 에센셜 립밤 15ml,에뛰드,"6,500","5,850",None
8698,14913,바디케어_립케어,https://image.oliveyoung.co.kr/uploads/images/...,https://www.oliveyoung.co.kr/store/goods/getGo...,A000000192917,[어워즈] 에뛰드 진저슈가 오버나이트 립 마스크 어워즈 한정기획 (2023),에뛰드,"7,000",NaN,None
8854,15069,바디케어_핸드케어,https://image.oliveyoung.co.kr/uploads/images/...,https://www.oliveyoung.co.kr/store/goods/getGo...,A000000168323,스킨알엑스랩 노티드 슈가 리치 핸드워시 300ml,스킨알엑스랩,"13,000",NaN,None


In [ ]:
# 제품명, 브랜드, 가격, 세일가, 리뷰 개수, 리뷰 평점, 
# 성분정보, 상세페이지 이미지

In [68]:
new_df.columns

Index(['product_code', 'category', 'images', 'LINK', 'goodsno'], dtype='object')

In [94]:
#추가 데이터 _plus에 저장
new_df.to_excel(r'D:\ODOC\jupyter_notebook\제품데이터_FULL\화장품 통합 데이터_유니크_V3_plus.xlsx',
                index=False)